How to handle double delimiter or multi delimiters

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession \
    .builder \
    .getOrCreate()

sc = spark.sparkContext

Read File

In [3]:
df = spark.read.csv('/home/phillipefs/spark_dev/pyspark-real-time-scenarios/data/multiples_delimiter.csv', header=True)
df.show()

+---+-------+---------+-----------+
| id|   name|      loc|      marks|
+---+-------+---------+-----------+
|  1|   ravi|bangalore|35|65|55|65|
|  2|    raj|  chennai|35|65|55|65|
|  3| mahesh|Hyderabad|35|65|55|65|
|  4| prasad|  chennai|35|65|55|65|
|  5|sridhar|     pune|35|65|55|65|
+---+-------+---------+-----------+



In [43]:
df.withColumn("marks_split", split(col("marks"), "\|"))\
    .withColumn("col1", col('marks_split')[0])\
    .withColumn("col2", col('marks_split')[1])\
    .withColumn("col3", col('marks_split')[2])\
    .withColumn("col4", col('marks_split')[3])\
    .show(truncate=False)

+---+-------+---------+-----------+----------------+----+----+----+----+
|id |name   |loc      |marks      |marks_split     |col1|col2|col3|col4|
+---+-------+---------+-----------+----------------+----+----+----+----+
|1  |ravi   |bangalore|35|65|55|65|[35, 65, 55, 65]|35  |65  |55  |65  |
|2  |raj    |chennai  |35|65|55|65|[35, 65, 55, 65]|35  |65  |55  |65  |
|3  |mahesh |Hyderabad|35|65|55|65|[35, 65, 55, 65]|35  |65  |55  |65  |
|4  |prasad |chennai  |35|65|55|65|[35, 65, 55, 65]|35  |65  |55  |65  |
|5  |sridhar|pune     |35|65|55|65|[35, 65, 55, 65]|35  |65  |55  |65  |
+---+-------+---------+-----------+----------------+----+----+----+----+



Function split and create columns 

In [46]:
def split_col_array_another_col(dataframe, col_name, delimiter):
    df_split = dataframe.withColumn('split', split(col(col_name),delimiter))
    max_elements = df_split.select(max(size('split'))).collect()[0][0]
    for i in range(max_elements):
        df_split = df_split.withColumn('col_'+str(i), col('split')[i])
    return df_split


split_col_array_another_col(df,col_name ='marks',delimiter='\|').show(truncate=False)

+---+-------+---------+-----------+----------------+----------------+-----+-----+-----+-----+
|id |name   |loc      |marks      |marks_split     |split           |col_0|col_1|col_2|col_3|
+---+-------+---------+-----------+----------------+----------------+-----+-----+-----+-----+
|1  |ravi   |bangalore|35|65|55|65|[35, 65, 55, 65]|[35, 65, 55, 65]|35   |65   |55   |65   |
|2  |raj    |chennai  |35|65|55|65|[35, 65, 55, 65]|[35, 65, 55, 65]|35   |65   |55   |65   |
|3  |mahesh |Hyderabad|35|65|55|65|[35, 65, 55, 65]|[35, 65, 55, 65]|35   |65   |55   |65   |
|4  |prasad |chennai  |35|65|55|65|[35, 65, 55, 65]|[35, 65, 55, 65]|35   |65   |55   |65   |
|5  |sridhar|pune     |35|65|55|65|[35, 65, 55, 65]|[35, 65, 55, 65]|35   |65   |55   |65   |
+---+-------+---------+-----------+----------------+----------------+-----+-----+-----+-----+

